# Homework 7 Link analysis

The url I choose is https://doc.scrapy.org/en/latest/intro/tutorial.html. 
I wrote a crawler to grab first 200 URLs. And this crawler considers the complications in '<a href'. More details will be commented in the code.

In [37]:
import numpy as np
import requests
from scrapy.http import TextResponse
from numpy import *

7.1 Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosin

In [38]:
#add url x to the list, updata matrix while crawling 
#fidx is the index of source url in urllist
#idx is the index of of next free page url in urllist
def addtolist(fidx,idx,x,url,matrix):
    i=x.rfind('#')  # remove the last '#something...', because it's just pointing to some section in the html page
    if i!=-1:
        x=x[:i]  # get rid of '#'
    if x not in urllist:  # a new page
        urllist.append(x)
        matrix[fidx][idx]=1
        idx+=1
    else:   # a old page
        k=urllist.index(x) # find the index of old page
        matrix[fidx][k]=1
    return idx


In [39]:
#crawl from url and update matrix if needed
#fidx is the index of url in urllist

def crawl(fidx,url,urllist,matrix):
    idx=len(urllist)
    i=url.rfind('/')
    me=url[0:i+1]  #current level 
    i=url.rfind('/',0,i-1)
    parent=url[0:i+1] # get the upper level
    i=url.find('://')
    r=requests.get(url)
    response= TextResponse(r.url, body=r.text, encoding='utf-8')
    link=response.xpath('//ul/li/a/@href').extract()  # all url link
    for x in link:
        if x.endswith('://'): # get rid of '://' illeagal format
            continue
        elif x.startswith('#'): # point to url itself, but same page
            matrix[fidx][fidx]=1
        elif x.startswith('http'):
            idx=addtolist(fidx,idx,x,urllist,matrix)
            if idx==200:
                return idx
        elif x.startswith('../'):
            x=parent+x[3:]
            idx=addtolist(fidx,idx,x,urllist, matrix)
            if idx==200:
                return idx
        else:
            x=me+x
            idx=addtolist(fidx,idx,x,urllist,matrix)
            if idx==200:
                return idx
    return idx

In [40]:
urllist=['https://doc.scrapy.org/en/latest/intro/tutorial.html']  #the starting url
matrix=np.zeros((200,200)) #the matrix
num=0
fidx=0
while num<200 and fidx<len(urllist):
    num=crawl(fidx,urllist[fidx],urllist,matrix)
    fidx+=1
print 'The first 200 URLs are:'
for x in urllist:
    print x    


The first 200 URLs are:
https://doc.scrapy.org/en/latest/intro/tutorial.html
https://doc.scrapy.org/en/latest/intro/overview.html
https://doc.scrapy.org/en/latest/intro/install.html
https://doc.scrapy.org/en/latest/intro/examples.html
https://doc.scrapy.org/en/latest/topics/commands.html
https://doc.scrapy.org/en/latest/topics/spiders.html
https://doc.scrapy.org/en/latest/topics/selectors.html
https://doc.scrapy.org/en/latest/topics/items.html
https://doc.scrapy.org/en/latest/topics/loaders.html
https://doc.scrapy.org/en/latest/topics/shell.html
https://doc.scrapy.org/en/latest/topics/item-pipeline.html
https://doc.scrapy.org/en/latest/topics/feed-exports.html
https://doc.scrapy.org/en/latest/topics/request-response.html
https://doc.scrapy.org/en/latest/topics/link-extractors.html
https://doc.scrapy.org/en/latest/topics/settings.html
https://doc.scrapy.org/en/latest/topics/exceptions.html
https://doc.scrapy.org/en/latest/topics/logging.html
https://doc.scrapy.org/en/latest/topics/stats

7.2  Create a Stochastic matrix from its resulting crawling

In [43]:
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    ret=1
    j=0
    while ret > 0.000001:  #until it converges
        rho1 = dot(rho,G)
        ret = np.linalg.norm(rho1-rho)  
        rho=rho1
        j+=1
    print 'pagerank converge after %d loops'%(j)
    return rho

In [48]:
page_matrix=matrix.copy()
for i in range(len(matrix)):
    k=sum(matrix[i])
    if k>0:
        page_matrix[i]/=k
        
H=pagerank(page_matrix)
print H

pagerank converge after 19 loops
[0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629 0.01169629
 0.01169629 0.01169629 0.01169629 0.00312836 0.00312376 0.00312376
 0.00309634 0.00329923 0.00309634 0.00309634 0.00309634 0.00309634
 0.00309634 0.00309634 0.00309634 0.00312836 0.00312836 0.00312836
 0.00311513 0.00311513 0.00311513 0.00312836 0.00312836 0.00312376
 0.00312376 0.00312836 0.00311935 0.00311935 0.00311935 0.00312376
 0.00353818 0.00312376 0.00312376 0.00312836 0.00312836 0.00312836
 0.00312836 0.00312836 0.00312836 0.00312836 0.00312836 0.00312836
 0.00312836 0.00310717 0.0031

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

In [49]:
v=np.argsort(-H)
print 'The top 5 page URL:'
for i in range(5):
    print urllist[v[i]]

The top 5 page URL:
https://doc.scrapy.org/en/latest/intro/tutorial.html
https://doc.scrapy.org/en/latest/topics/broad-crawls.html
https://doc.scrapy.org/en/latest/topics/firefox.html
https://doc.scrapy.org/en/latest/topics/firebug.html
https://doc.scrapy.org/en/latest/topics/leaks.html


7.4 For extra credit use the hits algorithm ( with a connectivity matrix 

In [46]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)    
    ret=1
    j=0
    while ret > 0.000001:  #until it converges
        a1 = dot(a,Au)
        a1= a1/sum(a1)
        h1 = dot(h,Hu)
        h1 = h1/ sum(h1)
        ret = np.linalg.norm(a1-a)+ np.linalg.norm(h1-h)   
        a=a1
        h=h1
        j+=1
    print 'Hits converge after %d loops'%(j)
    return a
        

In [50]:
H1=hits(matrix)
print H1

Hits converge after 5 loops
[0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356 0.02047356
 0.02047356 0.02047356 0.02047356 0.00048668 0.00048694 0.00048694
 0.00048901 0.00097672 0.00048901 0.00048901 0.00048901 0.00048901
 0.00048901 0.00048901 0.00048901 0.00048668 0.00048668 0.00048668
 0.00048772 0.00048772 0.00048772 0.00048668 0.00048668 0.00048694
 0.00048694 0.00048668 0.0004872  0.0004872  0.0004872  0.00048746
 0.00146289 0.00048694 0.00048694 0.00048668 0.00048668 0.00048668
 0.00048668 0.00048668 0.00048668 0.00048668 0.00048668 0.00048668
 0.00048668 0.00048797 0.00048797 

I found that if the data is not that big, the different between pagerank and hits algorithm is small. 

In [51]:
v1=np.argsort(-H1)
print 'The 5 top page URL:'
for i in range(5):
    print urllist[v1[i]]

The 5 top page URL:
https://doc.scrapy.org/en/latest/intro/tutorial.html
https://doc.scrapy.org/en/latest/topics/broad-crawls.html
https://doc.scrapy.org/en/latest/topics/firefox.html
https://doc.scrapy.org/en/latest/topics/firebug.html
https://doc.scrapy.org/en/latest/topics/leaks.html
